## Exploratory Data Analysis on New and Used Sedans

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from scipy import stats
import streamlit as st
import plotly.express as px

from IPython.display import display

In [2]:
# Load original DataFrame from .csv file.
df_og = pd.read_csv('../vehicles_us.csv')


In [3]:
# View DataFrame
display(df_og)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,nissan maxima,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37
51521,2700,2002.0,honda civic,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22
51522,3950,2009.0,hyundai sonata,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32
51523,7455,2013.0,toyota corolla,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71


In [4]:
# View DataFrame info
df_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [5]:
# Check for duplicates
print(df_og.duplicated().sum())
print()
print(df_og.duplicated(keep = False).sum())


0

0


There are no duplicate rows in this dataset.

In [6]:
# Calculate missing values of entire DataFrame.
print(df_og.isna().sum())

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64


There are several missing values in this dataset.  We could either decide to remove null values or we could decide to take a deeper look into the parts of the data where there are no null values.  Let's choose the latter for now.  

The model column has zero missing values but actually consists of both the make and the model component of the vehicle.  We can use the split() function separate the two compinents.

In [7]:
# fork original data frame into one that will be modified
df_a1 = df_og

# create make_n_model column from model column
df_a1['make_n_model'] = df_a1['model']

# remove original model column
df_a1 = df_a1.drop(['model'], axis=1)

# new data frame with split value columns
new = df_a1['make_n_model'].str.split(" ", n=1, expand=True)
 
# making separate make column from new data frame
df_a1['make'] = new[0]
 
# making separate model column from new data frame
df_a1['model'] = new[1]

# create separate make_n_type column using .agg()
df_a1['make_n_type'] = df_a1[['make', 'type']].agg(' '.join, axis=1)



In [8]:
display(df_a1)

,price,model_year,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make_n_model,make,model,make_n_type
0,9400,2011.0,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw x5,bmw,x5,bmw SUV
1,25500,NaN,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford f-150,ford,f-150,ford pickup
2,5500,2013.0,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai sonata,hyundai,sonata,hyundai sedan
3,1500,2003.0,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9,ford f-150,ford,f-150,ford pickup
4,14900,2017.0,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler 200,chrysler,200,chrysler sedan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37,nissan maxima,nissan,maxima,nissan sedan
51521,2700,2002.0,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22,honda civic,honda,civic,honda sedan
51522,3950,2009.0,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32,hyundai sonata,hyundai,sonata,hyundai sedan
51523,7455,2013.0,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71,toyota corolla,toyota,corolla,toyota sedan


In [9]:
df_a1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   condition     51525 non-null  object 
 3   cylinders     46265 non-null  float64
 4   fuel          51525 non-null  object 
 5   odometer      43633 non-null  float64
 6   transmission  51525 non-null  object 
 7   type          51525 non-null  object 
 8   paint_color   42258 non-null  object 
 9   is_4wd        25572 non-null  float64
 10  date_posted   51525 non-null  object 
 11  days_listed   51525 non-null  int64  
 12  make_n_model  51525 non-null  object 
 13  make          51525 non-null  object 
 14  model         51525 non-null  object 
 15  make_n_type   51525 non-null  object 
dtypes: float64(4), int64(2), object(10)
memory usage: 6.3+ MB


There is now both a make column and a model column.

In [10]:
print(sorted(list(df_a1['condition'].unique())))

['excellent', 'fair', 'good', 'like new', 'new', 'salvage']


In [11]:
# Create condition map to rank conditions numerically
conditions_map = {
    'new': 5,
    'like new': 4,
    'excellent': 3,
    'good': 2,
    'fair': 1,
    'salvage': 0,
}

In [12]:
# Define new function for condition map
def map_condition(condition: str) -> int:
    return conditions_map[condition]

In [13]:
df_a1['condition_rank'] = df_a1['condition'].apply(map_condition)
display(df_a1)

,price,model_year,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make_n_model,make,model,make_n_type,condition_rank
0,9400,2011.0,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw x5,bmw,x5,bmw SUV,2
1,25500,NaN,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford f-150,ford,f-150,ford pickup,2
2,5500,2013.0,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai sonata,hyundai,sonata,hyundai sedan,4
3,1500,2003.0,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9,ford f-150,ford,f-150,ford pickup,1
4,14900,2017.0,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler 200,chrysler,200,chrysler sedan,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37,nissan maxima,nissan,maxima,nissan sedan,4
51521,2700,2002.0,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22,honda civic,honda,civic,honda sedan,0
51522,3950,2009.0,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32,hyundai sonata,hyundai,sonata,hyundai sedan,3
51523,7455,2013.0,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71,toyota corolla,toyota,corolla,toyota sedan,2


Conditions are now ranked numerically from worst to best in ascending order.

In [14]:
print(sorted(list(df_a1['make'].unique())))

['acura', 'bmw', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'dodge', 'ford', 'gmc', 'honda', 'hyundai', 'jeep', 'kia', 'mercedes-benz', 'nissan', 'ram', 'subaru', 'toyota', 'volkswagen']


In [15]:
# Create map to catagorize each car brand by country
make_country_map = {
    'acura': 'japanese',
    'bmw': 'german',
    'buick': 'american',
    'cadillac': 'american',
    'chevrolet': 'american',
    'chrysler': 'american',
    'dodge': 'american',
    'ford': 'american',
    'gmc': 'american',
    'honda': 'japanese',
    'hyundai': 'korean',
    'jeep': 'american',
    'kia': 'korean',
    'mercedes-benz': 'german',
    'nissan': 'japanese',
    'ram': 'american',
    'subaru': 'japanese',
    'toyota': 'japanese',
    'volkswagen': 'german',
}

In [16]:
# Define new function for make country map
def map_make_country(make: str) -> str:
    return make_country_map[make]

In [17]:
df_a1['make_country'] = df_a1['make'].apply(map_make_country)
display(df_a1)

,price,model_year,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make_n_model,make,model,make_n_type,condition_rank,make_country
0,9400,2011.0,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw x5,bmw,x5,bmw SUV,2,german
1,25500,NaN,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford f-150,ford,f-150,ford pickup,2,american
2,5500,2013.0,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai sonata,hyundai,sonata,hyundai sedan,4,korean
3,1500,2003.0,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9,ford f-150,ford,f-150,ford pickup,1,american
4,14900,2017.0,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler 200,chrysler,200,chrysler sedan,3,american
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37,nissan maxima,nissan,maxima,nissan sedan,4,japanese
51521,2700,2002.0,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22,honda civic,honda,civic,honda sedan,0,japanese
51522,3950,2009.0,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32,hyundai sonata,hyundai,sonata,hyundai sedan,3,korean
51523,7455,2013.0,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71,toyota corolla,toyota,corolla,toyota sedan,2,japanese


The nationality of each car brand is now listed under 'make_country'.

In [18]:
df_a1['price'].max()

375000

The most expensive car in the dataset is priced at $375,000.

In [19]:
# Create a histogram of vehicle prices
fig1 = px.histogram(df_a1, x='price',
                  title="Distribution of Vehicle Prices")

# Show figure of plotly express histogram
fig1.show()

Relitive to the size of the dataset, there are not many vehicles priced over \$100,000.  Let's subset this data frame to only include vehicles that cost less than \$100,000.

In [20]:
df_a2 = df_a1.loc[df_a1['price'] < 100000]

In [21]:
# Create a histogram of vehicle prices
fig2 = px.histogram(df_a2, x='price',
                  title="Distribution of Vehicle Prices",
                  nbins=20)

# Show figure of plotly express histogram
fig2.show()

From the figure above, we can observe that the distribution of vehicle prices roughly follows an exponential distribution, with a plurality of vehicles being priced between \$5,000 and \$10,000.

In [22]:
df_sedan = df_a1.loc[df_a1['type'] == 'sedan']
display(df_sedan)

,price,model_year,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make_n_model,make,model,make_n_type,condition_rank,make_country
2,5500,2013.0,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai sonata,hyundai,sonata,hyundai sedan,4,korean
4,14900,2017.0,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler 200,chrysler,200,chrysler sedan,3,american
5,14990,2014.0,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15,chrysler 300,chrysler,300,chrysler sedan,3,american
6,12990,2015.0,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73,toyota camry,toyota,camry,toyota sedan,3,japanese
11,8990,2012.0,excellent,4.0,gas,111142.0,automatic,sedan,grey,NaN,2019-03-28,29,honda accord,honda,accord,honda sedan,3,japanese
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37,nissan maxima,nissan,maxima,nissan sedan,4,japanese
51521,2700,2002.0,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22,honda civic,honda,civic,honda sedan,0,japanese
51522,3950,2009.0,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32,hyundai sonata,hyundai,sonata,hyundai sedan,3,korean
51523,7455,2013.0,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71,toyota corolla,toyota,corolla,toyota sedan,2,japanese


Instead of looking at the entire datset of vehicles, we are specifically going to be analyzing sedans, so we created another subset of the data.

In [23]:
print(df_sedan.info())

<class 'pandas.core.frame.DataFrame'>
Index: 12154 entries, 2 to 51524
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           12154 non-null  int64  
 1   model_year      11306 non-null  float64
 2   condition       12154 non-null  object 
 3   cylinders       10898 non-null  float64
 4   fuel            12154 non-null  object 
 5   odometer        10253 non-null  float64
 6   transmission    12154 non-null  object 
 7   type            12154 non-null  object 
 8   paint_color     9965 non-null   object 
 9   is_4wd          563 non-null    float64
 10  date_posted     12154 non-null  object 
 11  days_listed     12154 non-null  int64  
 12  make_n_model    12154 non-null  object 
 13  make            12154 non-null  object 
 14  model           12154 non-null  object 
 15  make_n_type     12154 non-null  object 
 16  condition_rank  12154 non-null  int64  
 17  make_country    12154 non-null  obje

There are a lot of variables in which we are not interested.  Let's remove unwanted variables first.

In [24]:
# Create subset with only the columns we are interested in
df_sedan = df_sedan[['price', 'model_year', 'condition', 'odometer', 'condition_rank', 'make_country']].copy().reset_index()

print(df_sedan.info())

<class 'pandas.core.frame.DataFrame'>
Index: 12154 entries, 2 to 51524
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           12154 non-null  int64  
 1   model_year      11306 non-null  float64
 2   condition       12154 non-null  object 
 3   odometer        10253 non-null  float64
 4   condition_rank  12154 non-null  int64  
 5   make_country    12154 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 664.7+ KB
None


Now that we have removed unwanted variables, we can start removing the missing values that remain.  There are missing values in the 'model_year' and 'odometer' columns.

In [25]:
# Drop rows where 'model_year' and 'odometer' have missing values

# axis = 0 indicates rows are being dropped (axis = 1 would indicated columns are being dropped)
# how = "any" indicates to drop all rows with one or more missing values
df_sedan = df_sedan.dropna(axis = 0, how = "any").reset_index()

print(df_sedan.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9535 entries, 2 to 51523
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           9535 non-null   int64  
 1   model_year      9535 non-null   float64
 2   condition       9535 non-null   object 
 3   odometer        9535 non-null   float64
 4   condition_rank  9535 non-null   int64  
 5   make_country    9535 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 521.4+ KB
None


There are now no longer any missing values in the dataset, but the model year still has to be changed from float to integer.

In [26]:
# change model year from float to integer
df_sedan['model_year'] = df_sedan['model_year'].astype('int')

print(df_sedan.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9535 entries, 2 to 51523
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           9535 non-null   int64  
 1   model_year      9535 non-null   int64  
 2   condition       9535 non-null   object 
 3   odometer        9535 non-null   float64
 4   condition_rank  9535 non-null   int64  
 5   make_country    9535 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 521.4+ KB
None


All the data types now look correct.

In [27]:
df_sedan['price'].max()

64000

The highest priced sedan in the dataset is \$64,000.  In order to bin the price data in \$1,000 increments, we will have to create a histogram with 65 bins.

In [28]:
fig3 = px.histogram(df_sedan, 
                    x='price', 
                    color='make_country',
                    nbins=65)
fig3.show()

The vast majority of the sedans for sale come from Japanese or American brands and cost between \$3,000 and \$6,000.

In [29]:
fig4 = px.histogram(df_sedan, x='condition_rank', y='price',
             color='make_country', barmode='group',
             histfunc='avg', height=400,
             labels={'condition_rank': '0 = salvage, 1 = fair, 2 = good, 3 = excellent, 4 = like new, 5 = new'})
fig4.show()

Interesting takeaways: 

Salvage cars are on average more expensive than cars in fair condition.  The reasoning for this could be because salvage cars that are worth saving are more expensive than salvage cars not worth saving.  If a really cheap car had a salvage title, it would most likely go to the junk yard rather than the car lot.  

In most cases, cars are competitively price for their condition.  When it comes to new cars though, American brands are overcharging and likely to lose marketshare to Japanese, Korean, and German car brands.

In [30]:
fig5 = px.scatter(df_sedan, x='model_year', 
                  y='odometer', color='make_country', 
                  size='condition_rank',
                  labels={'model_year': 'Model Year', 
                      'odometer': 'Odometer Reading (in Miles)'})
fig5.show()

In [31]:
# Group the average 'price' of a vehicle by 'model_year' and 'make_country'
df_sedan_price_avg_by_year = df_sedan.groupby(['model_year', 'make_country'])['price'].mean().reset_index() 

In [32]:
fig6 = px.line(df_sedan_price_avg_by_year, 
              x='model_year', y='price', 
              color='make_country',
              labels={'model_year': 'Model Year', 
                      'price': 'Average Price'})
fig6.show()